In [1]:
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*Pyarrow will become a required dependency.*")
import pandas as pd
import geopandas as gpd
import numpy as np
import sys # For displaying package versions
import os # For managing directories and file paths if drive is mounted
import random
import json
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
# To reload submodules need to use this magic command to set autoreload on
# This command loads the autoreload extension, enabling the use of the %autoreload magic command.
# This sets autoreload to automatically reload all modules (both modules you've imported and your own modules) before executing a new line. 
# The value "2" means "always reload,"

%load_ext autoreload
%autoreload 2

In [3]:
# !pip install wget
# !pip install us
# !pip install seaborn
# !pip install fpdf2

In [4]:
from _lodes_data_structure import all_ods
from _lodes_data_structure import all_segparts
from _lodes_data_structure import all_charstems
from _lodes_data_structure import all_stems
from _lodes_data_structure import all_jobtypes
from _lodes_data_structure import all_mxjobtypes
from _lodes_data_structure import all_segstems
from lodes_datautil import *
from lodes_fullloop import *
from lodes_mcmcsa_util import *

In [ ]:
county_fips= "20057"

In [5]:
stacked_df = obtain_lodes_county_loop([county_fips], 
                    ['2020'], 
                    outputfoldername = "output",
                    ods = all_ods,
                    segparts = all_segparts,
                    jobtypes = all_jobtypes,
                    mxjobtypes = all_mxjobtypes,
                    segstems = all_segstems,
                    blocklist = '')

Importing and cleaning LODES data for the following files in the od group for 2020.
Importing and cleaning LODES data for the following files in the wac group for 2020.
Importing and cleaning LODES data for the following files in the rac group for 2020.


In [6]:
work_block_list = get_county_work_block_list(stacked_df, county_fips_code=county_fips, year='2020', od='od', seg='na')
home_block_list = get_county_home_block_list(stacked_df, county_fips_code=county_fips, year='2020', od='od', seg='na')
print("work block list: ", len(work_block_list))
print("home block list: ", len(home_block_list))

work block list:  323
home block list:  1364


In [8]:
seed_value = 1234
counter=1
for od in ["wac", "rac"]:
    joblist_df = pd.DataFrame()
    if od== "wac":
        block_list = work_block_list
    elif od == "rac":
        block_list = home_block_list
    
    for block in block_list[0:10]:
        seed_value += 1
        joblist = wac_rac_block_to_joblist(stacked_df = stacked_df, 
                                            block_fips= block ,
                                            years = ['2020'],
                                            seed_value = seed_value,
                                            outputfoldername = "output",
                                            od = od,
                                            reshape_vars = {'CE' : 'Earnings',
                                                                'CNS': 'IndustryCode',
                                                                'CA' : 'Age',
                                                                'CR' : 'Race',
                                                                'CT' : 'Ethnicity',
                                                                'CD' : 'Education',
                                                                'CS' : 'Sex'},
                                                            segstems = ['SE','SI','SA'])
        
        for (year, od), inner_dict in joblist.items():
            for key, df in inner_dict.items():
                df.to_csv(f'output2/{od}_{block}_joblist_{year}.csv', index=False)
                if counter == 1:
                    joblist_df = df.copy()  # Create a new DataFrame
                else:
                    joblist_df = pd.concat([joblist_df, df], ignore_index=True)   
        counter += 1
        print("*********************************************************************")

    joblist_df.to_csv(f'{od}_{county_fips}_joblist_{year}.csv', index=False)
    # display(wac_joblist_df)

Reshaping stacked joblists from block level data 
 to joblist with characteristics.
Errors in job counts...... values of mutual keys do not match
fixing the erros ..... iteration 1

All errors are fixed 
 all mutual keys match after 1 iterations.
Errors in job counts...... values of mutual keys do not match
fixing the erros ..... iteration 1

All errors are fixed 
 all mutual keys match after 1 iterations.
Errors in job counts...... values of mutual keys do not match
fixing the erros ..... iteration 1

All errors are fixed 
 all mutual keys match after 1 iterations.
Combing wac joblists.......
Checking jobs that appear in all three segment files for wac Files.
   For wac Age file includes:  4 Possible Jobs
   For wac Earnings+Age file includes:  8 Possible Jobs
   For wac SuperSector+Earnings+Age file includes:  12 Possible Jobs
11 of possible jobs exist in all three segments
Combining SuperSecore+Earnings+Age reduces possible joblists for wac files to 11 Jobs
*************************